<a href="https://colab.research.google.com/github/adhello007/PersonalProjects-DeepLearning/blob/main/LanguageTranslationSeq_SeqModels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive')

In [ ]:
!unzip fra-eng.zip

Archive:  fra-eng.zip
  inflating: _about.txt              
  inflating: fra.txt                 


In [ ]:
os.chdir('/content/drive/MyDrive/LanguageTranslation')

In [ ]:

#We're doing Character by Character translation


Lets Clean the Data

In [ ]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

with open('fra.txt', 'r',encoding='utf-8') as f:
  lines = f.read().split('\n')
for line in lines[: min(10000, len(lines)-1)]: #selecting the first 10000 lines -1
  input_text,target_text, _ = line.split('\t')
  target_text = '\t' + target_text + '\n'
  input_texts.append(input_text)
  target_texts.append(target_text)
  for char in input_text:
    if char not in input_characters:
      input_characters.add(char)
  for char in target_text:
    if char not in target_characters:
      target_characters.add(char)


In [ ]:
input_texts[:2], target_texts[:2], list(input_characters)[:2], list(target_characters)[:2]

(['Go.', 'Go.'], ['\tVa !\n', '\tMarche.\n'], ['g', 'o'], ['B', '\xa0'])

In [ ]:

input_texts[200]

'Beat it.'

In [ ]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters) #number of unique english chars
num_decoder_tokens = len(target_characters) #number of unique french chars
max_encoder_seq_length = max([len(txt) for txt in input_texts]) #longest sent in english
max_decoder_seq_length = max([len(txt) for txt in target_texts]) #longest sent in french

In [ ]:
num_encoder_tokens

70

In [ ]:
print('number of samples for training: ',len(input_texts))
print('number of unique english tokens: ', num_encoder_tokens)
print('number of unique french tokens: ', num_decoder_tokens)
print('longest english encoded length: ', max_encoder_seq_length)
print('longest deveoder frencgh length: ',max_decoder_seq_length)


number of samples for training:  10000
number of unique english tokens:  70
number of unique french tokens:  93
longest english encoded length:  14
longest deveoder frencgh length:  59


In [ ]:
#assiging each and every token an index
input_token_index = dict(
    [(char,i) for i, char in enumerate(input_characters)]
)

target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)]
)

In [ ]:
input_token_index

{' ': 0,
 '!': 1,
 '"': 2,
 '$': 3,
 '%': 4,
 '&': 5,
 "'": 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '0': 10,
 '1': 11,
 '2': 12,
 '3': 13,
 '5': 14,
 '7': 15,
 '8': 16,
 '9': 17,
 ':': 18,
 '?': 19,
 'A': 20,
 'B': 21,
 'C': 22,
 'D': 23,
 'E': 24,
 'F': 25,
 'G': 26,
 'H': 27,
 'I': 28,
 'J': 29,
 'K': 30,
 'L': 31,
 'M': 32,
 'N': 33,
 'O': 34,
 'P': 35,
 'Q': 36,
 'R': 37,
 'S': 38,
 'T': 39,
 'U': 40,
 'V': 41,
 'W': 42,
 'Y': 43,
 'a': 44,
 'b': 45,
 'c': 46,
 'd': 47,
 'e': 48,
 'f': 49,
 'g': 50,
 'h': 51,
 'i': 52,
 'j': 53,
 'k': 54,
 'l': 55,
 'm': 56,
 'n': 57,
 'o': 58,
 'p': 59,
 'q': 60,
 'r': 61,
 's': 62,
 't': 63,
 'u': 64,
 'v': 65,
 'w': 66,
 'x': 67,
 'y': 68,
 'z': 69}

In [ ]:
#now, one hot representation
import numpy as np

In [ ]:
#initially we're putting all values as 0
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')

decoder_input_data = np.zeros(
    (len(input_texts),max_decoder_seq_length,num_decoder_tokens)
) #using french instead of english

decoder_target_data = np.zeros(
    (len(input_texts),max_decoder_seq_length,num_decoder_tokens),dtype='float32'
)

In [ ]:
sample_encoder_input_data = np.zeros(
    (1, max_encoder_seq_length, num_encoder_tokens), dtype='float32')

sample_decoder_input_data = np.zeros(
    (1,max_decoder_seq_length,num_decoder_tokens)
)

sample_decoder_target_data = np.zeros(
    (1,max_decoder_seq_length,num_decoder_tokens),dtype='float32'
)

In [ ]:
encoder_input_data.shape

(10000, 14, 70)

In [ ]:
#now lets fill the above values

for i, (input_text,target_text) in enumerate(zip(input_texts,target_texts)): #i is the index for input and target text
  for t, char in enumerate(input_text): # t is the index for char in input_text
    encoder_input_data[i,t,input_token_index[char]] = 1 #since encodrer input data is already created, we're putting one where the char is present, rest its all 0s
  encoder_input_data[i, t+1:, input_token_index[' ']] = 1 #terminates the input_text properly
  for t, char in enumerate(target_text): #same done for target texat
    decoder_input_data[i, t, target_token_index[char]] = 1
    if t > 0:
      #decoder target data wil be ahead by one timestep
      # and will not include the start cahracter
      decoder_target_data[i, t-1, target_token_index[char]] =1
  decoder_input_data[i, t+1:, target_token_index[' ']] = 1
  decoder_target_data[i,t:, target_token_index[' ']] = 1



In [ ]:
input_text = "I am happy."
for t, char in enumerate(input_text): # t is the index for char in input_text
  sample_encoder_input_data[0,t,input_token_index[char]] = 1 #since encodrer input data is already created, we're putting one where the char is present, rest its all 0s
sample_encoder_input_data[0, t+1:, input_token_index[' ']] = 1 #terminates the input_text properly


In [ ]:
sample_encoder_input_data[0]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0

Create the LSTM Model

In [ ]:

from keras.models import Model
from keras.layers import Input, LSTM, Dense
batch_size = 64
epochs = 100
latent_dim = 256 #dimensionalituy of the encoding space

In [ ]:
#Define an input sequence and process it
encoder_inputs = Input(shape=(None,num_encoder_tokens)) # creating an input layer of the encoder None because input can have any length
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
#WE discard 'encoder_outputs' and only keep the states as we dont want it acc
#to the architecture
encoder_states = [state_h,state_c]

In [ ]:
#Set up the decdoer, using 'encoder_states' as initial state

decoder_inputs = Input(shape=(None,num_decoder_tokens))
#we set up our decoder to return full output sequences,
#and to return internal states as well.
#we dont use the return states in the training model, but we will use them
#in inference.

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _,_ = decoder_lstm(decoder_inputs,initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs= decoder_dense(decoder_outputs)



In [ ]:
#Define the model that will turn 'encoder input data' and 'decoder_input_data' into 'decoder_target_data'
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

#Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics =['accuracy'])
model.fit([encoder_input_data,decoder_input_data], decoder_target_data,batch_size=batch_size,epochs=epochs, validation_split=0.2)


Epoch 1/100
125/125 [==============================] - 11s 29ms/step - loss: 1.2127 - accuracy: 0.7327 - val_loss: 1.0695 - val_accuracy: 0.7133
Epoch 2/100
125/125 [==============================] - 1s 12ms/step - loss: 0.9373 - accuracy: 0.7484 - val_loss: 0.9617 - val_accuracy: 0.7335
Epoch 3/100
125/125 [==============================] - 1s 12ms/step - loss: 0.8413 - accuracy: 0.7676 - val_loss: 0.8664 - val_accuracy: 0.7606
Epoch 4/100
125/125 [==============================] - 1s 11ms/step - loss: 0.7453 - accuracy: 0.7919 - val_loss: 0.7753 - val_accuracy: 0.7766
Epoch 5/100
125/125 [==============================] - 1s 12ms/step - loss: 0.6693 - accuracy: 0.8081 - val_loss: 0.7309 - val_accuracy: 0.7861
Epoch 6/100
125/125 [==============================] - 2s 12ms/step - loss: 0.6276 - accuracy: 0.8181 - val_loss: 0.7033 - val_accuracy: 0.7940
Epoch 7/100
125/125 [==============================] - 1s 12ms/step - loss: 0.5982 - accuracy: 0.8250 - val_loss: 0.6488 - val_accuracy

To decode a test sentence, we will repeatedly:

1) Encode the input sentence and retrieve the initial decoder state
2) Run one step of the decoder with this initial state and a "start of sequence" token as target. The output will be the next target character.
3) Append the target character predicted and repeat.

In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape =(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state = decoder_states_inputs)
decoder_states = [state_h,state_c]
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)


In [ ]:
reverse_input_char_index = dict((i,char) for char,i in input_token_index.items())
reverse_target_char_index = dict((i,char) for char, i in target_token_index.items())
#reverse-looing up token index to decode sequences back to something readable


In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
for seq_index in range(100):
  #taking one seq for trying out decoding
  input_seq = encoder_input_data[seq_index: seq_index +1]
  decoded_sentence = decode_sequence(input_seq)
  print('-')
  print('Input sentence: ', input_texts[seq_index])
  print('decoded sentence: ', decoded_sentence)

1/1 [==============================] - 1s 687ms/step


KeyError: ignored